Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_r_kernel_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab_r_kernel:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/r_kernel).

Instalamos Rcpp:

In [33]:
install.packages("Rcpp",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/")

Warning message in install.packages("Rcpp", lib = "/usr/local/lib/R/site-library/", :
“installation of package ‘Rcpp’ had non-zero exit status”


In [54]:
install.packages("microbenchmark",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/")

La siguiente celda reiniciará el kernel de **IPython** para cargar los paquetes instalados en la celda anterior. Dar **Ok** en el mensaje que salga y continuar con el contenido del notebook.

In [2]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
library(Rcpp)
library(microbenchmark)

In [2]:
Rcf<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    sum_res<-0
    for(i in 1:n-1){
        x<-a+(i+1/2.0)*h_hat
        sum_res<-sum_res+f(x)
    }
    h_hat*sum_res
}

In [3]:
f<-function(x)exp(-x^2)

In [4]:
n<-10**6
aprox<-Rcf(f,0,1,n)


In [5]:
aprox

[1] 0.7468241

In [6]:
err_relativo<-function(aprox,obj)abs(aprox-obj)/abs(obj)

In [7]:
obj<-integrate(Vectorize(f),0,1)

In [8]:
err_relativo(aprox,obj$value)

[1] 6.719397e-14

In [9]:
system.time(Rcf(f,0,1,n))

   user  system elapsed 
  0.650   0.000   0.647 

In [10]:
cppFunction('double Rcf_Rcpp(double a, double b, int n){
             double h_hat;
             double sum_res=0;
             int i;
             double x;
             h_hat=(b-a)/n;
             for(i=0;i<=n-1;i++){
                    x = a+(i+1/2.0)*h_hat;
                    sum_res+=exp(-pow(x,2));
             }
             return h_hat*sum_res;
            }')

In [11]:
aprox_rcpp<-Rcf_Rcpp(0,1,n)

In [12]:
err_relativo(aprox_rcpp,obj$value)

[1] 6.719397e-14

In [13]:
system.time(Rcf_Rcpp(0,1,n))

   user  system elapsed 
  0.020   0.000   0.018 

In [14]:
mbk<-microbenchmark(
    Rcf(f,0,1,n),
    Rcf_Rcpp(0,1,n)
    )

In [15]:
print(mbk)

Unit: milliseconds
              expr       min        lq      mean    median        uq       max
   Rcf(f, 0, 1, n) 498.22033 543.54487 596.49554 570.83496 632.26109 844.79447
 Rcf_Rcpp(0, 1, n)  16.47919  17.22442  20.66479  18.27896  20.00948  99.21195
 neval
   100
   100
